In [ ]:
import habsim as hs
from datetime import datetime, timezone

In [ ]:
help(hs.MovingTarget)

In [ ]:
forecast = '''
ADDED LINE AT VALID 28/1500Z 17.5N  64.5W 

FORECAST VALID 29/0000Z 18.7N  65.7W
MAX WIND  65 KT...GUSTS  80 KT.
64 KT... 15NE   0SE   0SW   0NW.
50 KT... 30NE   0SE   0SW  10NW.
34 KT... 70NE  30SE   0SW  30NW.

FORECAST VALID 29/1200Z 20.5N  67.0W
MAX WIND  65 KT...GUSTS  80 KT.
64 KT... 15NE   0SE   0SW   0NW.
50 KT... 30NE   0SE   0SW  10NW.
34 KT... 70NE  30SE   0SW  30NW.

FORECAST VALID 30/0000Z 22.4N  68.4W
MAX WIND  70 KT...GUSTS  85 KT.
64 KT... 15NE   0SE   0SW   0NW.
50 KT... 30NE   0SE   0SW  10NW.
34 KT... 70NE  30SE   0SW  30NW.

FORECAST VALID 30/1200Z 24.2N  69.9W
MAX WIND  75 KT...GUSTS  90 KT.
64 KT... 20NE  10SE   0SW  10NW.
50 KT... 30NE  20SE  10SW  20NW.
34 KT... 70NE  50SE  30SW  50NW.

FORECAST VALID 31/1200Z 26.5N  74.0W
MAX WIND  90 KT...GUSTS 110 KT.
50 KT... 40NE  30SE  20SW  30NW.
34 KT... 80NE  70SE  40SW  60NW.

EXTENDED OUTLOOK. NOTE...ERRORS FOR TRACK HAVE AVERAGED NEAR 150 NM
ON DAY 4 AND 175 NM ON DAY 5...AND FOR INTENSITY NEAR 15 KT EACH DAY

OUTLOOK VALID 01/1200Z 27.7N  77.7W
MAX WIND 100 KT...GUSTS 120 KT.

OUTLOOK VALID 02/1200Z 28.6N  80.3W
MAX WIND 100 KT...GUSTS 120 KT.
'''

In [ ]:

def parse(forecast):
    year = datetime.utcnow().year
    lastday = 1
    month = datetime.utcnow().month
    traj = list()
    for line in forecast.split('\n'):
        if 'VALID' in line:
            line = line[line.find('VALID')+6:]
            day = int(line[0:2])
            if day < lastday: month += 1
            lastday = day
            hour = int(line[3:5])
            lat = float(line[9:13])
            lon = -float(line[16:20])
            timeobject = datetime(year, month, day, hour).replace(tzinfo=timezone.utc).timestamp()
            traj.append((timeobject, lat, lon))
    return list(zip(*traj))


In [ ]:
# 50 hours with waypoints at 5 hour intervals
profile = hs.ControlledProfile(80, 10)

# See docstring for argument details
profile.initialize(1000, 5000, 30000, seed=[79.0, 5000, 9000])
print(parse(forecast))
dorian = hs.MovingTarget(*parse(forecast))

pred = hs.Prediction(profile=profile, launchsite=hs.LaunchSite((37, -122)))
pred.profile = profile

In [ ]:
for i in range(20):
    pred.run(model=1)
    res = hs.util.optimize_step(pred, dorian, 5)
    print(res)
print(pred.profile.waypoints())
plt = hs.ioutil.WebPlot()
plt.origin(37, -122)
plt.add(pred.split(), colors=hs.ioutil.RandomColorGenerator())
plt.save('dorian.html')